In [1]:
#!pip install openai

In [3]:
#!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [3]:
#!pip install --upgrade transformers sentencepiece sacremoses

In [26]:
#!pip install sentence-transformers

In [1]:
import pandas as pd
import json
import openai
import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)

In [7]:
chat = pd.read_parquet('chat_history.parquet')

In [8]:
emb = pd.read_parquet('embedding.parquet')

In [18]:
emb.head(1)

,utterance,embedding
0,"""totalprice""","[-0.018974246457219124, 0.018123384565114975, 0.0051902648992836475, -0.00875679962337017, -0.02558261714875698, 0.015811871737241745, -0.03650202602148056, -0.029695121571421623, -0.003181873355060816, -0.021058861166238785, 0.014223594218492508, 0.02118649147450924, -0.018180107697844505, -0.007544319611042738, 0.02178209461271763, 0.0027156714349985123, 0.01813756488263607, 0.01084850449115038, 0.00011422394163673744, 0.0015191450947895646, -0.024646667763590813, 0.004151502624154091, -0.006572917569428682, -0.02315765619277954, -0.005094542633742094, -0.00885606650263071, 0.007501776330173016, -0.024533219635486603, -0.02284567430615425, 0.014407948590815067, 0.0016201850958168507, 0.0046655661426484585, -0.006331839598715305, -0.039082977920770645, -0.02814938686788082, -0.004584024660289288, 0.0017806083196774125, 0.00017958972603082657, 0.02013709396123886, 0.0038572459015995264, 0.023739080876111984, 0.000271655764663592, 0.004108959808945656, 0.00507681630551815, -0.04503902047872543, -0.0008774525485932827, 0.0315953828394413, -0.01613803580403328, -0.018917523324489594, -0.0006735999486409128, -0.014478853903710842, 0.038345564156770706, -0.03185064345598221, -0.01901678927242756, -0.004637203644961119, 0.005463249981403351, 0.024504857137799263, 0.019428040832281113, -0.004594660829752684, 0.019413858652114868, 0.007076344918459654, -0.0015785283176228404, -0.04634367674589157, -0.003640984883531928, -0.01591113954782486, -0.015131182037293911, -0.008934061974287033, 0.0020722062326967716, -0.012330424040555954, 0.00040726206498220563, -0.006689911242574453, 0.03624676913022995, 0.0003106536460109055, -0.03258805721998215, 0.026929816231131554, 0.01267786044627428, -0.0045485724695026875, 0.013670533895492554, 0.006984167732298374, 0.02392343431711197, 0.03088632971048355, -0.0024231872521340847, -0.004874736536294222, 0.005580243654549122, 0.02406524494290352, 0.02332782931625843, -0.014251956716179848, 0.02403688244521618, -0.015216268599033356, -0.01420232281088829, 0.012344605289399624, 0.002779486123472452, 0.0034708124585449696, 0.028447188436985016, -0.011444108560681343, 0.017499417066574097, -0.015982044860720634, 0.05014419928193092, -0.014890103600919247, -0.020746877416968346, ...]"


In [9]:
chat.head(1)

,processedUtterance,utterance,semantics,sqlForm,status,dataset,tags,createdAt,length
0,Customer_Monthly_Smart_View.totalprice,"""totalprice""","{'domain': 'Metric', 'filter': [], 'columns': ['Customer_Monthly_Smart_View.totalprice'], 'type': None, 'select': [('sum', 'currency/metrics/Customer_Monthly_Smart_View.totalprice')]}","SELECT round(sum(coalesce(subquery.""label_Customer_Monthly_Smart_View.totalprice"", 0)), 2) AS ""label_Customer_Monthly_Smart_View.totalprice"" \nFROM (SELECT round(sum(coalesce(@Customer_Monthly_Smart_View.totalprice, 0)), 2) AS ""label_Customer_Monthly_Smart_View.totalprice"" \nFROM (#)Customer_Monthly_Smart_View) AS subquery",success,3740fcb3-de4f-42cd-a14d-caba82b1cbcc,"[[""$Metrics"", [""\""totalprice\""""]]]",1675057540502,1


In [10]:
chat = chat[['utterance','tags']]

In [11]:
chat

,utterance,tags
0,"""totalprice""","[[""$Metrics"", [""\""totalprice\""""]]]"
1,+ve revenue,"[[""$UOps"", [""+ve""]], [""$Metrics"", [""revenue""]]]"
2,", on_hand_qty, customer, product number, product description, uom code, date","[[""$Metrics"", [""on_hand_qty""]], [""$Dimensions"", [""customer""]], [""$Dimensions"", [""product number""]], [""$Dimensions"", [""product description""]], [""$Dimensions"", [""uom code""]], [""$Datum"", [""date""]]]"
3,-ve revenue,"[[""$UOps"", [""-ve""]], [""$Metrics"", [""revenue""]]]"
4,1)\tlines per day for last 3 months,"[[""$MultiTableDate"", [""1)""]], [""$Dimensions"", [""lines""]], [""$Per"", [""per""]], [""$TrendPeriod"", [""day""]], [""$Filter"", [""for""]], [""$Period"", [""last"", ""3"", ""months""]]]"
...,...,...
3868,ytd spend by vendor top 5,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$Dimensions"", [""vendor""]], [""$Super"", [""top""]], [""$Number"", [""5""]]]"
3869,ytd spend by week,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$TrendPeriod"", [""week""]]]"
3870,ytd spend this calendar year,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Period"", [""this"", ""calendar year""]]]"
3871,ytd spend top 5 vendors,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Super"", [""top""]], [""$Number"", [""5""]], [""$Dimensions"", [""vendors""]]]"


In [12]:
#chat['utterance'] = chat['utterance'].str.replace(r'\d+\)', '', regex=True).str.replace('\t|\\\\', '', regex=True)
#chat['utterance'] = chat['utterance'].replace(', ', ' ', regex=True).replace('"', '', regex=True).replace('\t|\\\\', '', regex=True)
chat['utterance'] = chat['utterance'].replace('"', '', regex=True).replace('\t|\\\\', '', regex=True)
chat['utterance'] = chat['utterance'].str.strip(",")
chat['utterance'] = chat['utterance'].str.strip()

In [13]:
chat['tags'] = chat['tags'].str.replace('" ', '"')
chat['tags'] = chat['tags'].str.strip()

In [14]:
chat

,utterance,tags
0,totalprice,"[[""$Metrics"", [""\""totalprice\""""]]]"
1,+ve revenue,"[[""$UOps"", [""+ve""]], [""$Metrics"", [""revenue""]]]"
2,"on_hand_qty, customer, product number, product description, uom code, date","[[""$Metrics"", [""on_hand_qty""]], [""$Dimensions"", [""customer""]], [""$Dimensions"", [""product number""]], [""$Dimensions"", [""product description""]], [""$Dimensions"", [""uom code""]], [""$Datum"", [""date""]]]"
3,-ve revenue,"[[""$UOps"", [""-ve""]], [""$Metrics"", [""revenue""]]]"
4,1)lines per day for last 3 months,"[[""$MultiTableDate"", [""1)""]], [""$Dimensions"", [""lines""]], [""$Per"", [""per""]], [""$TrendPeriod"", [""day""]], [""$Filter"", [""for""]], [""$Period"", [""last"", ""3"", ""months""]]]"
...,...,...
3868,ytd spend by vendor top 5,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$Dimensions"", [""vendor""]], [""$Super"", [""top""]], [""$Number"", [""5""]]]"
3869,ytd spend by week,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$GroupBy"", [""by""]], [""$TrendPeriod"", [""week""]]]"
3870,ytd spend this calendar year,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Period"", [""this"", ""calendar year""]]]"
3871,ytd spend top 5 vendors,"[[""$Period"", [""ytd""]], [""$Metrics"", [""spend""]], [""$Super"", [""top""]], [""$Number"", [""5""]], [""$Dimensions"", [""vendors""]]]"


## Genrate paraphases

In [12]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
def random_state(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

random_state(42)

In [15]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [72]:
question = list(chat['utterance'])

q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",
           "Give me that", "Provide details on", "Tell me about", "Share information about", "Update me the",
           "Need info on", "Brief me on", "What is the"]

paraphrase = []
iter_ = 0
# chunked, n = question[494:500], 1

# for chunk in [chunked[i * n:(i + 1) * n] for i in range((len(chunked) + n - 1) // n )]:
for phrase in question[3500:]:
    contains_q_word = False
    for q_word in q_langs:
        if q_word in phrase:
            contains_q_word = True
            break

    if contains_q_word:
        input_phrase = phrase
    else:
        q_word = np.random.choice(q_langs)
        input_phrase = " ".join([q_word, phrase])

    para_phrases = parrot.augment(input_phrase= input_phrase,
                                use_gpu=False,
                                diversity_ranker="levenshtein",
                                do_diverse=True, 
                                max_return_phrases = 10, 
                                max_length=300, 
                                adequacy_threshold = 0.40, 
                                fluency_threshold = 0.40)

    para_sent = []
    if para_phrases is None:
        paraphrase.append('')
    else:
        for para in para_phrases:                                  
            para_sent.append(para[0])
        paraphrase.append(', '.join(para_sent))

    iter_ += 1
    if iter_ % 200 == 0:
        print(f"completed {iter_} items")

completed 200 items


In [74]:
len(paraphrase)

373

In [75]:
df1 = pd.DataFrame({'paraphrase':paraphrase})
df1

,paraphrase
0,"give me a supplier, give me the vendor, give me a vendor"
1,"provide details on the vendor contact page, give details on vendor contact, provide details on the vendor contact, give details on vendorcontact, provide details on vendor contact, provide details on vendorcontact"
2,"update vendor id, update vendorid, update my vendor id"
3,"bring me vendorpartnum, get me vendorpartnum"
4,"tell me about the vendorpo?, talk about vendorpo, tell me about the vendorpo, tell me more about vendorpo, tell me about vendorpo?, tell me about vendorpo"
...,...
368,"what are top 5 vendors ytd spends?, what are top 5 vendors' ytd spending?, what are top 5 vendors' ytd spend?, what are top 5 vendors ytd spend?"
369,"give details on the ytd spend by week, provide details on the ytd spend by week, provide details on ytd spend by week"
370,Tell me about ytd spend this calendar year
371,"please describe the top 5 vendors on ytd spend, please describe the top 5 vendors on the list, please describe the top 5 vendors on ytd, please tell me about the top 5 vendors"


In [15]:
df = pd.read_excel('paraphrased.xlsx')

In [77]:
df = pd.concat([df, df1], ignore_index=True)

In [81]:
df

,paraphrase
0,"give details about the total price, give details on the total price, give details on total price, provide details on the total price, provide details on total price"
1,"could you provide me with ve income, could you provide me with ve revenue, can you provide ve revenue?, could you provide some revenue?, could you provide revenue?, how can you provide ve revenue, could you provide ve revenue?"
2,"tell me the onhandqty customer product number product description uom code date?, show me the onhandqty customer product number product description uom code date?, tell me the onhandqty customer product number product description uom code date, what is onhandqty customer product number product description uom code date, what's the onhandqty customer product number product description uom code date, what is the onhandqty customer product number product description uom code date?, what is the onhandqty customer product number product description uom code date"
3,"need information on the revenue, need information on ve revenue, need information on revenue, need information on -ve revenues, need information on -ve revenue, need info on -ve revenue"
4,"tell me about 1 line a day for the last 3 months?, tell me about 1 line a day for the last 3 months, tell me about 1 line per day for the last 3 months?, tell me about 1 line per day for the last 3 months, tell me about 1line per day for the last 3 months"
...,...
3868,"what are top 5 vendors ytd spends?, what are top 5 vendors' ytd spending?, what are top 5 vendors' ytd spend?, what are top 5 vendors ytd spend?"
3869,"give details on the ytd spend by week, provide details on the ytd spend by week, provide details on ytd spend by week"
3870,Tell me about ytd spend this calendar year
3871,"please describe the top 5 vendors on ytd spend, please describe the top 5 vendors on the list, please describe the top 5 vendors on ytd, please tell me about the top 5 vendors"


In [79]:
df.to_excel('paraphrased.xlsx', index = False)

In [13]:
chat['paraphrase'] = df['paraphrase']

In [16]:
chat.head(1)

,utterance,tags,paraphrase
0,totalprice,"[[""$Metrics"", [""\""totalprice\""""]]]","give details about the total price, give details on the total price, give details on total price, provide details on the total price, provide details on total price"


In [17]:
# Create prompt and completion pairs
pairs = []
for index, ques in chat.iterrows():
    prompt = f"Question: {ques['utterance']} \Paraphrase: {ques['paraphrase']}"
    completion = f"Tags: {ques['tags']}"
    pairs.append({"prompt":prompt, "completion":completion})

In [18]:
# Export to JSON file
with open('final_pairs.json', 'w') as jf:
    json.dump(pairs, jf, indent=1)

In [19]:
with open('final_pairs.jsonl', 'w') as f:
    for pair in pairs:
        f.write(json.dumps(pair) + '\n')

### embadding

In [65]:
df1 = pd.read_excel('final_data.xlsx')

In [66]:
df1.shape

(3873, 2)

In [8]:
df1.head()

,utterance,paraphrase
0,totalprice,"give details about the total price, give details on the total price, give details on total price, provide details on the total price, provide details on total price"
1,+ve revenue,"could you provide me with ve income, could you provide me with ve revenue, can you provide ve revenue?, could you provide some revenue?, could you provide revenue?, how can you provide ve revenue, could you provide ve revenue?"
2,"on_hand_qty, customer, product number, product description, uom code, date","tell me the onhandqty customer product number product description uom code date?, show me the onhandqty customer product number product description uom code date?, tell me the onhandqty customer product number product description uom code date, what is onhandqty customer product number product description uom code date, what's the onhandqty customer product number product description uom code date, what is the onhandqty customer product number product description uom code date?, what is the onhandqty customer product number product description uom code date"
3,-ve revenue,"need information on the revenue, need information on ve revenue, need information on revenue, need information on -ve revenues, need information on -ve revenue, need info on -ve revenue"
4,1)lines per day for last 3 months,"tell me about 1 line a day for the last 3 months?, tell me about 1 line a day for the last 3 months, tell me about 1 line per day for the last 3 months?, tell me about 1 line per day for the last 3 months, tell me about 1line per day for the last 3 months"


In [4]:
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
import torch

In [25]:
#model_name = 'msmarco-distilbert-base-v4'
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
model = SentenceTransformer(model_name, device=torch_device)

In [9]:
df11 = df1.head()

In [27]:
# utterance_vectors = []
# batch_size = 64
# batch = []
# for row in tqdm(df1.itertuples()):
#     description =row.utterance
#     batch.append(description)
#     if len(batch) >= batch_size:
#         utterance_vectors.append(model.encode(batch))
#         batch = []

# if len(batch) > 0:
#     utterance_vectors.append(model.encode(batch))
#     batch = []

# vectors = np.concatenate(utterance_vectors)
# vectors.shape

In [40]:
utterance_vectors = []
for row in tqdm(df1.itertuples()):
    description =row.utterance
    utterance_vectors.append(model.encode(description))

0it [00:00, ?it/s]

In [ ]:
df1[df1['paraphrase'].isnull()]#.shape

In [56]:
df1['paraphrase'] = df1['paraphrase'].fillna('')

In [59]:
para_vectors = []
for row in tqdm(df1.itertuples()):
    description =row.paraphrase
    sen = description.split(',')
    para_ = []
    for s in sen:
        para_.append(model.encode(s))
    para_vectors.append(para_)

0it [00:00, ?it/s]

In [39]:
len(utterance_vectors[4])

384

In [62]:
len(para_vectors[0][0])

384

In [63]:
df1['utterance_emb'] = utterance_vectors
df1['paraphrase_emb'] = para_vectors

In [64]:
df1.head(1)

utterance  \
0  totalprice   

                                                                                                                                                             paraphrase  \
0  give details about the total price, give details on the total price, give details on total price, provide details on the total price, provide details on total price   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         utterance_emb  \
0  [1.239465e-05, 0.023100434, -0.018898621, -0.0046917275, -0.059613224, -0.03898938, 0.042270295, 0.11776524, 0.074706666, 0.040492382, 0.032677364, -0.040878844, -0.0078399945, 0.009263733, -0.008205963, 0.030697908, 0.040175997, -0.0071107605, -0.10568312, -0.04188914, -0.049506564, -0.07267244, -0.028211283, -0.005321294, 0.07237665, 0.08059314, 0.005633787, 0.024946736, 0.065832585, 0.021558186, -0.06967778, 0.042800687, 0.07881511, -0.07175112, 0.05525981, -0.09782893, -0.056527704, 0.027202144, -0.053356152, 0.049043663, -0.008305837, -0.05106206, -0.10977852, -0.01565476, -0.004137913, -0.01678138, -0.006171615, 0.072478876, 0.06683653, 0.102945305, 0.022325268, 0.0042813797, -0.036560092, 0.0035883274, 0.053280804, -0.05390082, -0.008821404, -0.03209273, 0.010621838, -0.0070576523, -0.017126907, -0.030752389, -0.02388984, 0.039094422, 0.14425384, -0.034058932, 0.041576445, -0.010596893, -0.04711764, 0.041406054, 0.021924129, 0.048492447, 0.051639464, -0.048626076, 0.026404886, -0.015528862, 0.15151915, -0.034131978, 0.014053866, -0.022791587, 0.04472535, -0.089069024, -0.06515232, -0.041649174, 0.0001774244, 0.02485707, 0.08499074, -0.010503007, 0.037111793, -0.024629446, -0.046494987, -0.03172179, -0.07186385, -0.033378348, -0.0911621, 0.1163163, -0.04410818, -0.077048525, 0.006640728, 0.11869003, ...]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [71]:
para_vectors = []
para_ = []
for row in tqdm(df1.itertuples()):
    description = row.paraphrase
    if description is not None:
        sen = description.split(',')
        for s in sen:
            para_.append(s)
            para_vectors.append(model.encode(s))

0it [00:00, ?it/s]

In [73]:
df2 = pd.DataFrame({
    'paraphrase': para_,
    'embedding': para_vectors
})

In [74]:
df2.head(1)

,paraphrase,embedding
0,give details about the total price,"[0.01893174, 0.07348564, -0.038605474, 0.016451959, -0.028498594, -0.03418547, 0.05790684, 0.13909923, 0.051803127, 0.040033724, -0.0063241743, -0.08565104, 0.00637484, -0.03996353, -0.058740616, -0.003068075, 0.028021699, -0.015593983, -0.04402353, -0.07623192, -0.07376969, 0.0014121693, -0.00017240277, 0.020248126, -0.014222971, 0.054566626, -0.0032540592, 0.0043743053, 0.039601617, -0.030673282, -0.033652816, 0.015202267, 0.032967534, 0.030292664, -0.041373737, -0.09527751, -0.009276832, 0.00059683324, 0.0077838595, -0.0021596157, -0.013816504, -0.046077844, -0.08634276, 0.0055227005, 0.07333058, 0.02763602, 0.000527042, 0.038939387, 0.0948691, 0.08999077, 0.0017911609, -0.0076073683, -0.018842498, 0.017278286, 0.039071433, -0.009420801, -0.08326968, -0.026879163, 0.069829196, 0.020954687, 0.026988, -0.019208848, -0.05698333, 0.05880349, 0.10022165, -0.1096897, -0.041242022, -0.073734276, 0.009029805, -0.031889662, 0.028459081, 0.0035541707, 0.07371635, -0.05856431, -0.048814274, -0.028156051, 0.08768032, 0.0036289073, -0.002323049, 0.0150135895, 0.024673345, -0.075623766, -0.05977668, 0.00026026086, 0.058232423, -0.002741481, 0.03655679, 0.09225535, -0.009480164, -0.04374721, 0.073452026, -0.020556612, -0.106712185, -0.005801683, -0.041255247, 0.05460917, 0.027580578, 0.02152291, 0.050415702, 0.047065407, ...]"


In [91]:
df2.to_parquet('embedding_para.parquet', engine='pyarrow')

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

In [78]:
sample_query = df2.iloc[12345].paraphrase
print(sample_query)

 give details on revenue for the last year by month


In [79]:
query_vector = model.encode(sample_query)  

In [80]:
scores = cosine_similarity([query_vector], para_vectors)[0]

In [81]:
top_scores_ids = np.argsort(scores)[-5:][::-1]

In [83]:
for top_id in top_scores_ids:
    print("New_story: ")
    print(df2.iloc[top_id].paraphrase)
    print("-------------------------------------------------------------------------------------")

New_story: 
 give details on revenue for the last year by month
-------------------------------------------------------------------------------------
New_story: 
 give details on revenue for last year by month
-------------------------------------------------------------------------------------
New_story: 
 give details of revenue for the last year by month
-------------------------------------------------------------------------------------
New_story: 
 provide details on revenue for the last year by month
-------------------------------------------------------------------------------------
New_story: 
 provide details on revenue for last year by month
-------------------------------------------------------------------------------------


In [90]:
np.sort(scores)[::-1][:6]

array([1.0000001 , 0.9958489 , 0.9945643 , 0.9856138 , 0.97443086,
       0.9501984 ], dtype=float32)

In [85]:
np.argsort(scores)[-5:]

array([12348, 12347, 12344, 12346, 12345])

## Parrot

In [ ]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def random_state(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

random_state(42)

In [ ]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [10]:
phrases = ["totalprice",
           "lines per day for last 3 months",
           "ytd spend by vendor top 5",
           "ytd totalprice",
           "on_hand_qty customer product number product description uom code date",
           "ytd spend this calendar year"
]

In [35]:
# for phrase in phrases:
#     print("Input_phrase: ", phrase)
#     para_phrases = parrot.augment(input_phrase=phrase)
#     for para_phrase in para_phrases:
#         print(para_phrase)

In [11]:
for phrase in phrases:
    print("-"*100)
    print("Input_phrase: ", phrase)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=phrase,
                                use_gpu=False,
                                diversity_ranker="levenshtein",
                                do_diverse=True, 
                                max_return_phrases = 10, 
                                max_length=300, 
                                adequacy_threshold = 0.50, 
                                fluency_threshold = 0.50)
    for paraphrase in para_phrases:                                  
        print(paraphrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  totalprice
----------------------------------------------------------------------------------------------------
('total cost', 17)
('the total cost', 14)
('total price', 13)
('totalprice', 12)
('the total price', 10)
----------------------------------------------------------------------------------------------------
Input_phrase:  lines per day for last 3 months
----------------------------------------------------------------------------------------------------
('the last 3 months', 27)
('for the last 3 months', 26)
('lines per day for the last three months', 21)
('1 line per day for the last 3 months', 16)
----------------------------------------------------------------------------------------------------
Input_phrase:  ytd spend by vendor top 5
----------------------------------------------------------------------------------------------------
('yearly sales by vendor t

In [48]:
import numpy as np
np.random.seed(0)
q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",
           "Give me that", "Provide details on", "Tell me about", "Share information about", "Update me the",
           "Need info on", "Brief me on", "What is the"]

for phrase in phrases:
    print("*"*100)
    contains_q_word = False
    for q_word in q_langs:
        if q_word in phrase:
            contains_q_word = True
            break
    
    if contains_q_word:
        input_phrase = phrase
    else:
        # add a random question word to the phrase
        q_word = np.random.choice(q_langs)
        input_phrase = " ".join([q_word, phrase])
    
    print("Input_phrase: ", input_phrase)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase= input_phrase,
                                use_gpu=False,
                                diversity_ranker="levenshtein",
                                do_diverse=True, 
                                max_return_phrases = 10, 
                                max_length=300, 
                                adequacy_threshold = 0.50, 
                                fluency_threshold = 0.50)
    for paraphrase in para_phrases:                                  
        print(paraphrase)

****************************************************************************************************
Input_phrase:  What is the totalprice
----------------------------------------------------------------------------------------------------
('tell me the total price of the product?', 31)
('tell me the total price of the goods?', 31)
('tell me the total price of the item?', 30)
('tell me the total cost?', 23)
('is it a total price?', 21)
('is there a total price?', 21)
('tell me the total price?', 19)
('tell me the total price', 18)
("what's the total price?", 16)
('what is the total price?', 14)
****************************************************************************************************
Input_phrase:  Give me that lines per day for last 3 months
----------------------------------------------------------------------------------------------------
('what lines per day for the last 3 months?', 26)
('give me those lines per day for the last 3 months', 19)
****************************

In [61]:
import numpy as np
np.random.seed(0)
q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",
           "Give me that", "Provide details on", "Tell me about", "Share information about", "Update me the",
           "Need info on", "Brief me on", "What is the"]

for phrase in phrases:
    print("*"*100)
    contains_q_word = False
    for q_word in q_langs:
        if q_word in phrase:
            contains_q_word = True
            break
    
    if contains_q_word:
        input_phrase = phrase
    else:
        # add a random question word to the phrase
        q_word = np.random.choice(q_langs)
        input_phrase = " ".join([q_word, phrase])
    
    print("Input_phrase: ", input_phrase)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase= input_phrase,
                                use_gpu=False,
                                diversity_ranker="jaccard",
                                do_diverse=True, 
                                max_return_phrases = 10, 
                                max_length=300, 
                                adequacy_threshold = 0.30, 
                                fluency_threshold = 0.30)
    for paraphrase in para_phrases:                                  
        print(paraphrase)

****************************************************************************************************
Input_phrase:  What is the totalprice
----------------------------------------------------------------------------------------------------


AttributeError: 'NoneType' object has no attribute 'items'

## Test

In [3]:
# import torch
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
# model_name = 'tuner007/pegasus_paraphrase'
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
# def get_response(input_text,num_return_sequences,num_beams):
#     batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60).to(torch_device)
#     translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
#     tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
#     return tgt_text

# context = "on_hand_qty customer product number product description uom code date"
# num_return_sequences = 5
# num_beams = 10
# get_response(context, num_return_sequences, num_beams)

In [ ]:
# def get_response(input_text, num_return_sequences, num_beams):
#     input_tensor = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0).to(torch_device)
#     batch = {"input_ids": input_tensor, "attention_mask": torch.ones_like(input_tensor)}
#     translated = model.generate(**batch, max_length=100, num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
#     tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
#     return tgt_text

# context = "on_hand_qty customer product number product description uom code date"
# num_return_sequences = 5
# num_beams = 10
# get_response(context, num_return_sequences, num_beams)


In [28]:
# model_name = 'tuner007/pegasus_paraphrase'
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

# def get_response(input_text,num_return_sequences,num_beams):
#     batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
#     translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
#     tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
#     return tgt_text


In [29]:
# num_beams = 10
# num_return_sequences = 5
# context = "ytd spend this calendar year"
# get_response(context,num_return_sequences,num_beams)

## Pegasus

In [12]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, PegasusTokenizerFast, AutoTokenizer, AutoModelForSeq2SeqLM

In [13]:
model1 = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer1 = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

In [17]:
def get_paraphrased_sentences(model_, tokenizer_, sentence, num_return_sequences=5, num_beams=5):
    inputs = tokenizer_([sentence], truncation=True, padding="longest", return_tensors="pt")
    outputs = model_.generate(
        **inputs,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
      )
    return tokenizer_.batch_decode(outputs, skip_special_tokens=True)

In [18]:
sentence = "on_hand_qty customer product number product description uom code date"
get_paraphrased_sentences(model1, tokenizer1, sentence, num_beams=10, num_return_sequences=5)

['On_hand_qty customer product number',
 'On_hand_qty customer product number.',
 'The customer product number is on hand.',
 'The customer product number and description is on hand.',
 'The customer product number is described in the code date.']

In [6]:
get_paraphrased_sentences(model1, tokenizer1, 'total price', num_beams=10, num_return_sequences=5)

['There is a total price.',
 'The total price.',
 'The total price is.',
 'The price is total.',
 'The total price']

In [20]:
phrases = ["totalprice",
           "lines per day for last 3 months",
           "ytd spend by vendor top 5",
           "ytd totalprice",
           "on_hand_qty customer product number product description uom code date",
           "ytd spend this calendar year"
]
for phas in phrases:
    print('Input Phrases :', phas )
    print(get_paraphrased_sentences(model1, tokenizer1, phas, num_beams=10, num_return_sequences=5))

Input Phrases : totalprice
['total price', 'The total price.', 'There is a total price.', 'There is a totalprice.', 'total price.']
Input Phrases : lines per day for last 3 months
['There were lines per day for the last 3 months.', 'For the last 3 months, there were lines per day.', 'There were lines per day for 3 months.', 'There were lines per day for the last three months.', 'For the last 3 months, lines per day.']
Input Phrases : ytd spend by vendor top 5
['Spend by vendor top 5.', 'Spend by vendor top 5', 'Spending by vendor top 5', 'Spending by vendor top 5.', 'Spend by vendor top 5']
Input Phrases : ytd totalprice
['Total price.', 'Ytd total price.', 'The total price.', 'ytd total price', 'Total price']
Input Phrases : on_hand_qty customer product number product description uom code date
['On_hand_qty customer product number', 'On_hand_qty customer product number.', 'The customer product number is on hand.', 'The customer product number and description is on hand.', 'The custome

In [21]:
tokenizer2 = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model2 = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [34]:
sentence = "ytd spend this calendar year"
get_paraphrased_sentences(model2, tokenizer2, "paraphrase: " + sentence, num_beams=10, num_return_sequences=5)

['ytd spend this calendar year',
 'This calendar year ytd spend',
 'ytd spent this calendar year',
 'ytd spend this calendar year.',
 'Ytd spend this calendar year']

In [22]:
phrases = ["totalprice",
           "lines per day for last 3 months",
           "ytd spend by vendor top 5",
           "ytd totalprice",
           "on_hand_qty customer product number product description uom code date",
           "ytd spend this calendar year"
]
for phas in phrases:
    print('Input Phrases :', phas )
    print(get_paraphrased_sentences(model2, tokenizer2, phas, num_beams=10, num_return_sequences=5))

Input Phrases : totalprice
['Totalprice totalprice totalprice totalprice totalprice totalprice total', 'Totalprice totalprice totalprice', 'Totalprice totalprice totalprice totalprice totalprice totalprice', 'Totalprice totalprice totalprice totalprice', 'Totalprice totalprice totalprice totalprice totalprice']
Input Phrases : lines per day for last 3 months
['Lines per day for last 3 months for last 3 months per day for last 3 months', 'Lines per day for the last 3 months for last 3 months per day for last 3 months', 'Lines per day for the last 3 months for last 3 months.', 'Lines per day for last 3 months for last 3 months.', 'Lines per day for last 3 months for last 3 months per day for last 3 months ']
Input Phrases : ytd spend by vendor top 5
['ytd spend by vendor top 5', 'ytd spend by vendor top 5 ytd spend by vendor', 'ytd spend by vendor top 5 top 5 ytd spend by vendor', 'ytd spend by vendor top 5 top 5 ytd spend by vendor ', 'ytd spend by vendor top 5 ytd spend by vendor.']
In

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

In [24]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=300
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [25]:
text = 'What is the totalprice'
paraphrase(text)

['What is the aggregate cost?',
 'Can you tell me the overall cost of living?',
 'How much does it cost in total?',
 'What is the overall value of the goods or services sold?',
 'How much is the total cost?']

In [26]:
phrases = ["totalprice",
           "lines per day for last 3 months",
           "ytd spend by vendor top 5",
           "ytd totalprice",
           "on_hand_qty customer product number product description uom code date",
           "ytd spend this calendar year"
]
q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",
           "Give me that", "Provide details on", "Tell me about", "Share information about", "Update me the",
           "Need info on", "Brief me on", "What is the"]

for phrase in phrases:
    print("*"*100)
    contains_q_word = False
    for q_word in q_langs:
        if q_word in phrase:
            contains_q_word = True
            break
    
    if contains_q_word:
        input_phrase = phrase
    else:
        # add a random question word to the phrase
        q_word = np.random.choice(q_langs)
        input_phrase = " ".join([q_word, phrase])
    
    print("Input_phrase: ", input_phrase)
    print("-"*100)                                  
    print(paraphrase(input_phrase))

****************************************************************************************************
Input_phrase:  Update me the totalprice
----------------------------------------------------------------------------------------------------
["Can you tell me the amount of money I'll receive?", 'What is the total value of all items in your possession?', 'How much is the total cost?', 'Could you inform me of the total cost?', 'What is the total value of all items in your possession? Request']
****************************************************************************************************
Input_phrase:  Get me lines per day for last 3 months
----------------------------------------------------------------------------------------------------
['Have you been receiving daily lines for the past three months?', 'Give me daily lines for the past three months.', 'I have received daily lines for the past three months.', 'Have you been receiving daily lines for the past three months? Request'

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')


In [ ]:
sentences = ['What is the totalprice']

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

In [37]:
def longest_substring(s):
    if not s:
        return 0
    start = maxLength = 0
    usedChars = {}
    for i in range(len(s)):
        if s[i] in usedChars and start <= usedChars[s[i]]:
            start = usedChars[s[i]] + 1
        else:
            maxLength = max(maxLength, i - start + 1)
        usedChars[s[i]] = i
    return maxLength


In [40]:
s = "pwwekdfff"
print(longest_substring(s))  # Output: 3


5


In [40]:
import numpy as np 

q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",           "Give me that", "Provide details on", "Tell me about", "Share the data", "Update me the",           "Need info on", "Brief me on", "What is the"]

phrases = ["totalprice",           "lines per day for last 3 months",           "ytd spend by vendor top 5",           "brief me ytd totalprice",           "Could you provide me on_hand_qty customer product number product description uom code date",           "ytd spend this calendar year"]

for phrase in phrases:
    print("-" * 100)
    if any(q_word.lower() in phrase.lower() for q_word in q_langs):
        full_phrase = phrase
    else:
        add_str = np.random.choice(q_langs)
        full_phrase = " ".join([add_str, phrase])
    print("Input_phrase: ", full_phrase)


----------------------------------------------------------------------------------------------------
Input_phrase:  can you get me totalprice
----------------------------------------------------------------------------------------------------
Input_phrase:  Provide details on lines per day for last 3 months
----------------------------------------------------------------------------------------------------
Input_phrase:  Update me the ytd spend by vendor top 5
----------------------------------------------------------------------------------------------------
Input_phrase:  can you give me brief me ytd totalprice
----------------------------------------------------------------------------------------------------
Input_phrase:  Could you provide me on_hand_qty customer product number product description uom code date
----------------------------------------------------------------------------------------------------
Input_phrase:  Brief me on ytd spend this calendar year


In [46]:
import numpy as np 

q_langs = ["can you give me", "can you get me", "give me", "Could you provide", "Get me",           "Give me that", "Provide details on", "Tell me about", "Share the data", "Update me the",           "Need info on", "Brief me on", "What is the"]

phrases = ["totalprice",           "lines per day for last 3 months",           "ytd spend by vendor top 5",           "brief me ytd totalprice",           "Could you provide me on_hand_qty customer product number product description uom code date",           "ytd spend this calendar year"]

for phrase in phrases:
    print("-" * 100)
    add_str = np.random.choice([q for q in q_langs if not phrase.lower().startswith(q.lower())])
    full_phrase = f"{add_str} {phrase}"
    print("Input_phrase: ", full_phrase)


----------------------------------------------------------------------------------------------------
Input_phrase:  What is the totalprice
----------------------------------------------------------------------------------------------------
Input_phrase:  Give me that lines per day for last 3 months
----------------------------------------------------------------------------------------------------
Input_phrase:  Update me the ytd spend by vendor top 5
----------------------------------------------------------------------------------------------------
Input_phrase:  Share the data brief me ytd totalprice
----------------------------------------------------------------------------------------------------
Input_phrase:  can you give me Could you provide me on_hand_qty customer product number product description uom code date
----------------------------------------------------------------------------------------------------
Input_phrase:  can you give me ytd spend this calendar year
